In [4]:
import os, sys, wget

In [5]:
CUSTOM_MODEL_NAME = 'centernet_hourglass104' 
PRETRAINED_MODEL_NAME = 'centernet_hg104_1024x1024_kpts_coco17_tpu-32'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_hg104_1024x1024_kpts_coco17_tpu-32.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [6]:
paths = {
    'WORKSPACE_PATH': os.path.join('TensorFlow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('TensorFlow','scripts'),
    'APIMODEL_PATH': os.path.join('TensorFlow','models'),
    'ANNOTATION_PATH': os.path.join('TensorFlow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('TensorFlow', 'workspace','images'),
    'MODEL_PATH': os.path.join('TensorFlow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('TensorFlow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('TensorFlow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('TensorFlow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('TensorFlow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('TensorFlow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('TensorFlow','protoc')
 }

In [7]:
files = {
    'PIPELINE_CONFIG':os.path.join('TensorFlow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [4]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

In [8]:
#!pip install wget
import wget

In [9]:
import object_detection

In [7]:
#Download the pre_trained_model and move it to \Tensorflow\workspace\pre-trained-models, then unzip it.
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

--2022-12-01 00:15:57--  http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_hg104_1024x1024_kpts_coco17_tpu-32.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.19.80, 2a00:1450:4005:80b::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.19.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1453412998 (1,4G) [application/x-tar]
Saving to: ‘centernet_hg104_1024x1024_kpts_coco17_tpu-32.tar.gz’

centernet_hg104_102 100%[===================>]   1,35G  52,3MB/s    in 27s     

2022-12-01 00:16:25 (51,1 MB/s) - ‘centernet_hg104_1024x1024_kpts_coco17_tpu-32.tar.gz’ saved [1453412998/1453412998]

centernet_hg104_1024x1024_kpts_coco17_tpu-32/
centernet_hg104_1024x1024_kpts_coco17_tpu-32/checkpoint/
centernet_hg104_1024x1024_kpts_coco17_tpu-32/checkpoint/ckpt-0.data-00000-of-00001
centernet_hg104_1024x1024_kpts_coco17_tpu-32/checkpoint/checkpoint
centernet_hg104_1024x1024_kpts_coco17_tpu-32

In [16]:
#os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME) = 'Tensorflow\\workspace\\annotations\\label_map.pbtxt 
#--> already done, also don't need to do it again!

#labels = [{'id':1, 'name':'building'}, {'id':2, 'name':'tree'}, {'id':3, 'name':'sky'}, {'id':4, 'name':'car'}, {'id':5, 'name':'sign'}, {'id':6, 'name':'road'}, {'id':7, 'name':'pedestrian'}, {'id':8, 'name':'fence'}, {'id':9, 'name':'pole'}, {'id':10, 'name':'sidewalk'}, {'id':11, 'name':'bicyclist'}]
labels = [{'name':'Car', 'id':1}, {'name':'Van', 'id':2}, {'name':'Truck', 'id':3}, {'name':'Pedestrian', 'id':4},{'name':'Person_sitting', 'id':5}, {'name':'Cyclist', 'id':6}, {'name':'Tram', 'id':7}, {'name':'Misc', 'id':8}, {'name':'DontCare', 'id':9}]

with open(os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME), 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [13]:
!cd TensorFlow/models/research/object_detection/dataset_tools

In [15]:
!dir

project_object_detection  subs	TensorFlow  train_model.ipynb


In [9]:
!python TensorFlow/models/research/object_detection/dataset_tools/create_kitti_tf_record.py \
        --data_dir=TensorFlow/workspace/images \
        --output_path=TensorFlow/workspace/annotations/kitti \

2022-12-01 00:43:31.140914: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-01 00:43:31.730179: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-01 00:43:31.730217: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-01 00:43:33.335446: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

In [10]:
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

In [11]:
import object_detection
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [12]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [13]:
config

{'model': center_net {
   num_classes: 90
   feature_extractor {
     type: "hourglass_104"
     channel_means: 104.01361846923828
     channel_means: 114.03422546386719
     channel_means: 119.91659545898438
     channel_stds: 73.60276794433594
     channel_stds: 69.89082336425781
     channel_stds: 70.91507720947266
     bgr_ordering: true
   }
   image_resizer {
     keep_aspect_ratio_resizer {
       min_dimension: 1024
       max_dimension: 1024
       pad_to_max_dimension: true
     }
   }
   object_detection_task {
     task_loss_weight: 1.0
     offset_loss_weight: 1.0
     scale_loss_weight: 0.10000000149011612
     localization_loss {
       l1_localization_loss {
       }
     }
   }
   object_center_params {
     object_center_loss_weight: 1.0
     classification_loss {
       penalty_reduced_logistic_focal_loss {
         alpha: 2.0
         beta: 4.0
       }
     }
     min_box_overlap_iou: 0.699999988079071
     max_box_predictions: 100
   }
   keypoint_label_map_path: 

In [14]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [17]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 100
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0.index')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'kitti_train.tfrecord')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'kitti_val.tfrecord')]

In [18]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:
    f.write(config_text)

In [19]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [20]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=10000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

In [61]:
print(command)

python TensorFlow/models/research/object_detection/model_main_tf2.py --model_dir=TensorFlow/workspace/models/centernet_hourglass104 --pipeline_config_path=TensorFlow/workspace/models/centernet_hourglass104/pipeline.config --num_train_steps=10000


In [55]:
files['PIPELINE_CONFIG']

'TensorFlow/workspace/models/centernet_hourglass104/pipeline.config'

In [64]:
import sys

In [63]:
!python TensorFlow/models/research/object_detection/model_main_tf2.py --model_dir=TensorFlow/workspace/models/centernet_hourglass104 --pipeline_config_path=TensorFlow/workspace/models/centernet_hourglass104/pipeline.config --num_train_steps=10000

2022-12-03 11:26:58.194512: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-03 11:26:58.194550: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/user/anaconda3/envs/tf2od/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/home/user/anaconda3/envs/tf2od/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/home/user/anaconda3/envs/tf2od/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl5mutexC1Ev']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/home/user/anaconda3/envs/tf2od/lib/python